In [85]:
########################################################
# Author: Izzat Zanail - izzat.zanail@gmail.com
# Language: Python 3.12
# Created: 2024-03-21
# Updated: 2024-08-18
# Project: Malaysia Super League Data Analysis
# Description: MSL Football Clubs Data Scraping and Analysis from Transfermarkt website
########################################################

In [86]:
# Import libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math

In [87]:
# Extract team page links for each season

season_year = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
trsfrmkt_msl_link = 'https://www.transfermarkt.com/malaysia-super-league/startseite/wettbewerb/MYS1/plus/?saison_id='

YearList = []
TeamsList = []
TeamSquadSize = []
TeamNoForeigners = []
TeamMarketValue = []
TeamLinksList = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

for year in range(0, len(season_year)):
    page_year = trsfrmkt_msl_link + str(season_year[year])
    pageTree = requests.get(page_year, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
    
    Teams = pageSoup.find_all('td', {'class': 'hauptlink no-border-links'})
    SquadSize = pageSoup.find_all('td', {'class': 'zentriert'})
    NoForeigners = pageSoup.find_all('td', {'class': 'zentriert'})
    MarketValue = pageSoup.find_all('td', {'class': 'rechts'})
    TeamLinks = pageSoup.find_all('td', {'class': 'hauptlink no-border-links'})
    
    for i in range(0, len(Teams)):
        str_Teams = str(Teams[i]).split('title="',1)[1].split('">',1)[0]
        if 'Johor Darul' in str_Teams:
            str_Teams = 'Johor Darul Takzim FC'
            TeamsList.append(str_Teams)
        else:
            TeamsList.append(str_Teams)
    for i in range(4, len(Teams)*4+4, 4):
        str_SquadSize = str(SquadSize[i]).split('>', 2)[2].split('<')[0]
        TeamSquadSize.append(str_SquadSize)
    for i in range(6, len(Teams)*4+6, 4):
        str_NoForeigners = str(NoForeigners[i]).split('>')[1].split('<')[0]
        TeamNoForeigners.append(str_NoForeigners)
    for i in range(3, len(Teams)*2+2, 2):
        str_MarketValue = str(MarketValue[i]).split('€')[1].split('<')[0]
        if 'k' in str_MarketValue:
            str_mv = str_MarketValue.split('k')[0]
            flt_mv = float(str_mv)*1000
            TeamMarketValue.append(flt_mv)
        elif 'm' in str_MarketValue:
            str_mv = str_MarketValue.split('m')[0]
            flt_mv = float(str_mv)*1000000
            TeamMarketValue.append(flt_mv)
        else:
            TeamMarketValue.append(float(flt_mv))
    for i in range(0, len(TeamLinks)):
        str_TeamLinks = str(TeamLinks[i]).split('a href="',1)[1].split('"',1)[0]
        TeamLinksList.append("https://www.transfermarkt.com" + str_TeamLinks)
    for i in range(0, len(Teams)):
        YearList.append(str(season_year[year] + 1))

In [88]:
# Create Dataframe for the extracted links

df_msl_teams = pd.DataFrame({
                    "SEASON":YearList,
                    "TEAM":TeamsList,
                    "SQUAD_SIZE":TeamSquadSize,
                    "NO_OF_FOREIGNERS":TeamNoForeigners,
                    "MARKET_VALUE_EUR":TeamMarketValue,
                    "LINK":TeamLinksList
})
# set dtypes for each column
df_msl_teams['SEASON'] = df_msl_teams['SEASON'].astype(str)
df_msl_teams['TEAM'] = df_msl_teams['TEAM'].astype(str)
df_msl_teams['SQUAD_SIZE'] = df_msl_teams['SQUAD_SIZE'].astype(int)
df_msl_teams['NO_OF_FOREIGNERS'] = df_msl_teams['NO_OF_FOREIGNERS'].astype(int)
df_msl_teams['LINK'] = df_msl_teams['LINK'].astype(str)

# remove duplicated rows of Year 2025
df_msl_teams = df_msl_teams[df_msl_teams['SEASON'] != '2025']

df_msl_teams.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 0 to 98
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SEASON            99 non-null     object 
 1   TEAM              99 non-null     object 
 2   SQUAD_SIZE        99 non-null     int32  
 3   NO_OF_FOREIGNERS  99 non-null     int32  
 4   MARKET_VALUE_EUR  99 non-null     float64
 5   LINK              99 non-null     object 
dtypes: float64(1), int32(2), object(3)
memory usage: 4.6+ KB


In [89]:
df_msl_teams.head(n=10)

,SEASON,TEAM,SQUAD_SIZE,NO_OF_FOREIGNERS,MARKET_VALUE_EUR,LINK
0,2017,Johor Darul Takzim FC,36,9,6630000.0,https://www.transfermarkt.com/johor-darul-tazi...
1,2017,Melaka United FC,52,10,3130000.0,https://www.transfermarkt.com/melaka-united-fc...
2,2017,Kedah FA,33,3,3030000.0,https://www.transfermarkt.com/kedah-darul-aman...
3,2017,Selangor FC,35,8,2980000.0,https://www.transfermarkt.com/selangor-fc/star...
4,2017,Sri Pahang FC,33,5,2850000.0,https://www.transfermarkt.com/sri-pahang-fc/st...
5,2017,Selangor FC II,38,7,2830000.0,https://www.transfermarkt.com/selangor-fa-ii/s...
6,2017,Kelantan FC,45,9,2730000.0,https://www.transfermarkt.com/kelantan-fa/star...
7,2017,Terengganu FC II,36,6,2580000.0,https://www.transfermarkt.com/terengganu-fc-ii...
8,2017,FELDA United FC,35,8,2250000.0,https://www.transfermarkt.com/felda-united-fc/...
9,2017,Penang FC,41,8,2250000.0,https://www.transfermarkt.com/penang-fc/starts...


In [90]:
df_msl_teams.tail(n=10)

,SEASON,TEAM,SQUAD_SIZE,NO_OF_FOREIGNERS,MARKET_VALUE_EUR,LINK
89,2024,Sabah FC,34,8,3890000.0,https://www.transfermarkt.com/sabah-fc/startse...
90,2024,Sri Pahang FC,26,5,3800000.0,https://www.transfermarkt.com/sri-pahang-fc/st...
91,2024,Kedah Darul Aman FC,40,5,3750000.0,https://www.transfermarkt.com/kedah-darul-aman...
92,2024,Kuala Lumpur City FC,29,5,3380000.0,https://www.transfermarkt.com/kuala-lumpur-cit...
93,2024,Perak FC,28,8,3030000.0,https://www.transfermarkt.com/perak-fc/startse...
94,2024,Kuching City,37,10,2920000.0,https://www.transfermarkt.com/kuching-city/sta...
95,2024,Kelantan Darul Naim,38,10,2610000.0,https://www.transfermarkt.com/kelantan-united/...
96,2024,PDRM FC,35,9,2550000.0,https://www.transfermarkt.com/pdrm-fa/startsei...
97,2024,Negeri Sembilan FC,30,8,2480000.0,https://www.transfermarkt.com/negeri-sembilan-...
98,2024,Penang FC,30,5,2450000.0,https://www.transfermarkt.com/penang-fc/starts...


In [91]:
# Filter dataframe by TEAM
df_msl_teams[df_msl_teams['TEAM'] == 'Johor Darul Takzim FC']

,SEASON,TEAM,SQUAD_SIZE,NO_OF_FOREIGNERS,MARKET_VALUE_EUR,LINK
0,2017,Johor Darul Takzim FC,36,9,6630000.0,https://www.transfermarkt.com/johor-darul-tazi...
12,2018,Johor Darul Takzim FC,38,13,11780000.0,https://www.transfermarkt.com/johor-darul-tazi...
24,2019,Johor Darul Takzim FC,30,6,7400000.0,https://www.transfermarkt.com/johor-darul-tazi...
36,2020,Johor Darul Takzim FC,33,7,6780000.0,https://www.transfermarkt.com/johor-darul-tazi...
48,2021,Johor Darul Takzim FC,36,8,6680000.0,https://www.transfermarkt.com/johor-darul-tazi...
60,2022,Johor Darul Takzim FC,37,10,8580000.0,https://www.transfermarkt.com/johor-darul-tazi...
72,2023,Johor Darul Takzim FC,37,10,9930000.0,https://www.transfermarkt.com/johor-darul-tazi...
86,2024,Johor Darul Takzim FC,35,15,10830000.0,https://www.transfermarkt.com/johor-darul-tazi...


In [92]:

max_foreigners = df_msl_teams.loc[df_msl_teams['NO_OF_FOREIGNERS'].idxmax()]
min_foreigners = df_msl_teams.loc[df_msl_teams['NO_OF_FOREIGNERS'].idxmin()]

max_team_size = df_msl_teams.loc[df_msl_teams['SQUAD_SIZE'].idxmax()]
min_team_size = df_msl_teams.loc[df_msl_teams['SQUAD_SIZE'].idxmin()]

highest_value = df_msl_teams.loc[df_msl_teams['MARKET_VALUE_EUR'].idxmax()]
lowest_value = df_msl_teams.loc[df_msl_teams['MARKET_VALUE_EUR'].idxmin()]



In [93]:
# Team with MOST import players of all time
df_msl_teams[df_msl_teams['NO_OF_FOREIGNERS'] == max_foreigners[3]]

C:\Users\izzat\AppData\Local\Temp\ipykernel_19808\4195141080.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_msl_teams[df_msl_teams['NO_OF_FOREIGNERS'] == max_foreigners[3]]


,SEASON,TEAM,SQUAD_SIZE,NO_OF_FOREIGNERS,MARKET_VALUE_EUR,LINK
86,2024,Johor Darul Takzim FC,35,15,10830000.0,https://www.transfermarkt.com/johor-darul-tazi...


In [94]:
# Team with LEAST import players of all time
df_msl_teams[df_msl_teams['NO_OF_FOREIGNERS'] == min_foreigners[3]]

C:\Users\izzat\AppData\Local\Temp\ipykernel_19808\406950331.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_msl_teams[df_msl_teams['NO_OF_FOREIGNERS'] == min_foreigners[3]]


,SEASON,TEAM,SQUAD_SIZE,NO_OF_FOREIGNERS,MARKET_VALUE_EUR,LINK
59,2021,Petaling Jaya City FC,29,0,1540000.0,https://www.transfermarkt.com/petaling-jaya-ci...
70,2022,Petaling Jaya City FC,28,0,2360000.0,https://www.transfermarkt.com/petaling-jaya-ci...


In [95]:
# Team with BIGGEST Squad Size of all time
df_msl_teams[df_msl_teams['SQUAD_SIZE'] == max_team_size[2]]

C:\Users\izzat\AppData\Local\Temp\ipykernel_19808\3997120412.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_msl_teams[df_msl_teams['SQUAD_SIZE'] == max_team_size[2]]


,SEASON,TEAM,SQUAD_SIZE,NO_OF_FOREIGNERS,MARKET_VALUE_EUR,LINK
84,2023,Kelantan FC,56,10,2500000.0,https://www.transfermarkt.com/kelantan-fa/star...


In [96]:
# Team with SMALLEST Squad Size of all time
df_msl_teams[df_msl_teams['SQUAD_SIZE'] == min_team_size[2]]

C:\Users\izzat\AppData\Local\Temp\ipykernel_19808\545737929.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_msl_teams[df_msl_teams['SQUAD_SIZE'] == min_team_size[2]]


,SEASON,TEAM,SQUAD_SIZE,NO_OF_FOREIGNERS,MARKET_VALUE_EUR,LINK
90,2024,Sri Pahang FC,26,5,3800000.0,https://www.transfermarkt.com/sri-pahang-fc/st...


In [97]:
# HIGEST VALUE Team of all time
df_msl_teams[df_msl_teams['MARKET_VALUE_EUR'] == highest_value[4]]

C:\Users\izzat\AppData\Local\Temp\ipykernel_19808\2498566889.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_msl_teams[df_msl_teams['MARKET_VALUE_EUR'] == highest_value[4]]


,SEASON,TEAM,SQUAD_SIZE,NO_OF_FOREIGNERS,MARKET_VALUE_EUR,LINK
12,2018,Johor Darul Takzim FC,38,13,11780000.0,https://www.transfermarkt.com/johor-darul-tazi...


In [98]:
# LOWEST VALUE Team of all time
df_msl_teams[df_msl_teams['MARKET_VALUE_EUR'] == lowest_value[4]]

C:\Users\izzat\AppData\Local\Temp\ipykernel_19808\4085969809.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_msl_teams[df_msl_teams['MARKET_VALUE_EUR'] == lowest_value[4]]


,SEASON,TEAM,SQUAD_SIZE,NO_OF_FOREIGNERS,MARKET_VALUE_EUR,LINK
47,2020,Sabah FC,29,6,450000.0,https://www.transfermarkt.com/sabah-fc/startse...


In [99]:
# Average Team Value Per Season
avg_team_value = df_msl_teams.pivot_table(values='MARKET_VALUE_EUR', index='TEAM', aggfunc='mean')
avg_team_value.fillna(value=0).sort_values(by='MARKET_VALUE_EUR', ascending=False)

,MARKET_VALUE_EUR
TEAM,
Johor Darul Takzim FC,8.576250e+06
Kedah Darul Aman FC,4.070000e+06
Selangor FC,3.901250e+06
Kedah FA,3.472500e+06
Sri Pahang FC,3.260000e+06
Terengganu FC,3.197143e+06
Kuching City,3.195000e+06
Sabah FC,3.084000e+06
Kuala Lumpur City FC,3.071667e+06


In [100]:
# No of import players Per Team, Per Season
no_import_players = df_msl_teams.pivot_table(values='NO_OF_FOREIGNERS', index='TEAM', columns='SEASON')
no_import_players.fillna(value=0)

SEASON,2017,2018,2019,2020,2021,2022,2023,2024
TEAM,,,,,,,,
FELDA United FC,8.0,0.0,9.0,5.0,0.0,0.0,0.0,0.0
Johor Darul Takzim FC,9.0,13.0,6.0,7.0,8.0,10.0,10.0,15.0
Kedah Darul Aman FC,0.0,0.0,0.0,0.0,5.0,7.0,9.0,5.0
Kedah FA,3.0,8.0,9.0,6.0,0.0,0.0,0.0,0.0
Kelantan Darul Naim,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
Kelantan FC,9.0,11.0,0.0,0.0,0.0,0.0,10.0,0.0
Kelantan United,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
Kuala Lumpur City FC,0.0,5.0,5.0,0.0,4.0,4.0,8.0,5.0
Kuching City,0.0,0.0,0.0,0.0,0.0,0.0,14.0,10.0


In [101]:
# Export the DataFrame to csv file

df_msl_teams.to_csv(r"C:\Users\izzat\OneDrive\Desktop\MalaysiaSuperLeague_TransfermrktDataScraping\datacsv\MSL_TeamPages.csv", index=False)

In [108]:
# Create lists

SeasonList = []
PlayerTeamsList = []
PlayersList = []
NumberList = []
AgeList = []
PositionList = []
NationList = []
ValueList = []
grouped_positionList = []
cleaned_values = []

# Scrape player data, from links that were previously scraped

for link in range(0, len(TeamLinksList)):
    page = TeamLinksList[link]
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
    
    TeamNames = pageSoup.find_all("h1", {"class": "data-header__headline-wrapper data-header__headline-wrapper--oswald"})
    Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
    Numbers = pageSoup.find_all("div", {"class": "rn_nummer"})
    Age = pageSoup.find_all("td", {"class": "zentriert"})
    Positions = pageSoup.find_all("table", {"class": "inline-table"})
    Nationality = pageSoup.find_all('img', {'class': 'flaggenrahmen'})
    Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
    
    season_year = str(int(TeamLinksList[link].split("/", 8)[8]))
    
    #if season 2024, skip 3 rows, else, skip 4 rows 
    if season_year == "2024":
        r = 3
    else:
        r = 4
    
    for i in range(0, len(Players)):
        SeasonList.append(season_year)
        #if season_year == "2024":
        #    SeasonList.append("2024/2025") # Starting from year 2024, Malaysia Super League will start in May 2024 and end in 2025.
        #else:
        #    SeasonList.append(season_year) 
    
    for i in range(0, len(Players)):
        str_Team = str(TeamNames).split('>')[1].split('\n            ')[1].split('        </h1')[0]
        #if 'Johor Darul' in str_Teams:
        #    str_Teams = 'Johor Darul Takzim FC'
        #    PlayerTeamsList.append(str_Teams)
        #else:
        #    PlayerTeamsList.append(str_Teams)
        PlayerTeamsList.append(str_Team)
    
    for i in range(0, len(Players)):
        str_Players = str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1]
        PlayersList.append(str_Players)
    
    for i in range(0, len(Numbers)):
        str_Numbers = str(Numbers[i]).split('class="rn_nummer">',1)[1].split('</div>',1)[0]
        NumberList.append(str_Numbers)
           
    for i in range(0, len(Positions)):
        str_Position = str(Positions[i]).split('<td>', 1)[1].split('</td>', 1)[0].split('\n ', 1)[1].split('           ', 1)[1].split('        ',1)[0]
        PositionList.append(str_Position)
    
    # Convert specific position names into a more general name - Marked as "Position Type"
    for j in range(0, len(Positions)):
        if 'Striker' in PositionList[j]:
            grouped_positionList.append('Forwards')
        elif 'Forward' in PositionList[j]:
            grouped_positionList.append('Forwards')
        elif 'Midfield' in PositionList[j]:
            grouped_positionList.append('Midfielders')
        elif 'Midfielder' in PositionList[j]:
            grouped_positionList.append('Midfielders')
        elif 'Winger' in PositionList[j]:
            grouped_positionList.append('Midfielders')
        elif 'Back' in PositionList[j]:
            grouped_positionList.append('Defenders')
        elif 'Defender' in PositionList[j]:
            grouped_positionList.append('Defenders')
        else:
            grouped_positionList.append(PositionList[j])
    
    #for i in range(2, (len(Players)*r), r): 
    #    str_Nationality = str(Nationality[i]).split('" class',1)[0].split('<img alt="',1)[1]
    #    NationList.append(str_Nationality)

    for i in range(1, (len(Players)*r), r):
        str_Age = str(Age[i]).split("(",1)[1].split(")",1)[0]
        AgeList.append(str_Age)
    
    for i in range(0, len(Values)):
        ValueList.append(Values[i].text)
        
# Data cleaning - change "Value" datatype from string type to float type
for a in range(0, len(ValueList)):
    if 'k' in ValueList[a]:
        str_a = str(ValueList[a]).split('€')[1].split('k')[0]
        flt_a = float(str_a)*1000
        cleaned_values.append(flt_a)
    elif 'm' in ValueList[a]:
        str_a = str(ValueList[a]).split('€')[1].split('m')[0]
        flt_a = float(str_a)*1000000
        cleaned_values.append(flt_a)
    else:
        cleaned_values.append(float(a))


In [122]:
# Create DataFrame for full extracted data

df_msl_player = pd.DataFrame({
                         "SEASON":SeasonList,
                         "TEAM":PlayerTeamsList,
                         "PLAYER":PlayersList,
                         "NUMBER":NumberList,
                         "AGE":AgeList,
                         "POSITION":PositionList,
                         "POSITION_TYPE":grouped_positionList,
                         #"NATIONALITY":NationList,
                         "VALUE_EUR":ValueList,
                         "VALUE_EUR_1":cleaned_values                         
                        })
df_msl_player

# clean AGE
df_msl_player['AGE'] = np.where(df_msl_player['AGE'] == '-', int(0), df_msl_player['AGE'])
df_msl_player['AGE'] = df_msl_player['AGE'].astype('int')

df_msl_player = df_msl_player[df_msl_player['AGE'] != 0]

# remove duplicated rows of Year 2025 & reset index
df_msl_player = df_msl_player.drop_duplicates().reset_index()

# remove column 'index'
df_msl_player = df_msl_player.drop(columns=['index'])

# clean VALUE_EUR
df_msl_player['VALUE_EUR'] = np.where(df_msl_player['VALUE_EUR'] == '-', '€0n', df_msl_player['VALUE_EUR'])

df_msl_player['VALUE_EUR_CLEANED'] = ''
for i in range(0, len(df_msl_player)):
    val = df_msl_player.loc[i, 'VALUE_EUR'].split('€')[1]

    res_k = 'k' in val
    res_m = 'm' in val
    res_nothing = 'n' in val

    if res_k:
        new_val = float(val.split('k')[0]) * 1000
    elif res_m:
        new_val = float(val.split('m')[0]) * 1000000
    elif res_nothing:
        new_val = float(val.split('n')[0])*0

    df_msl_player.loc[i, 'VALUE_EUR_CLEANED'] = new_val

df_msl_player['VALUE_EUR'] = df_msl_player['VALUE_EUR_1']

# remove column 'VALUE_EUR_CLEANED'
df_msl_player = df_msl_player.drop(columns=['VALUE_EUR_CLEANED', 'VALUE_EUR_1'])

df_msl_player.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3434 entries, 0 to 3433
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SEASON         3434 non-null   object 
 1   TEAM           3434 non-null   object 
 2   PLAYER         3434 non-null   object 
 3   NUMBER         3434 non-null   object 
 4   AGE            3434 non-null   int32  
 5   POSITION       3434 non-null   object 
 6   POSITION_TYPE  3434 non-null   object 
 7   VALUE_EUR      3434 non-null   float64
dtypes: float64(1), int32(1), object(6)
memory usage: 201.3+ KB


In [124]:
# Export the DataFrame to csv file

df_msl_player.to_csv(r"C:\Users\izzat\OneDrive\Desktop\MalaysiaSuperLeague_TransfermrktDataScraping\datacsv\MSL_FullData.csv", index=False)

In [125]:
max_age = df_msl_player.loc[df_msl_player['AGE'].idxmax()]
min_age = df_msl_player.loc[df_msl_player['AGE'].idxmin()]

print(max_age.iloc[4])
print(min_age.iloc[4])

41
15


In [126]:
# YOUNGEST Player of all time
df_msl_player[df_msl_player['AGE'] == min_age.iloc[4]]

,SEASON,TEAM,PLAYER,NUMBER,AGE,POSITION,POSITION_TYPE,VALUE_EUR
32,2016,Johor Darul Ta'zim,Hadi Fayyadh,-,15,Centre-Forward,Forwards,32.0
543,2017,Kelantan FC,Nik Umar Nik Aziz,41,15,Centre-Back,Midfielders,545.0
2894,2022,Kelantan FC,Mohd Afif,-,15,Goalkeeper,Defenders,2910.0
2909,2022,Kelantan FC,Farish Ainun,96,15,Centre-Back,Midfielders,2925.0
2928,2022,Kelantan FC,Haikal Pauzi,-,15,Left Winger,Goalkeeper,2944.0
2946,2022,Kelantan FC,Izaham Haslin,97,15,Centre-Forward,Defenders,2962.0


In [127]:
# YOUNGEST Player in 2024
season_2024 = df_msl_player[df_msl_player['SEASON'] == '2024']
season_2024

youngest_2024 = season_2024.loc[season_2024['AGE'].idxmin()]

season_2024[season_2024['AGE'] == youngest_2024[4]]

C:\Users\izzat\AppData\Local\Temp\ipykernel_19808\561479257.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  season_2024[season_2024['AGE'] == youngest_2024[4]]


,SEASON,TEAM,PLAYER,NUMBER,AGE,POSITION,POSITION_TYPE,VALUE_EUR
2987,2024,Johor Darul Ta'zim,Christian Abad,12,18,Goalkeeper,Defenders,3003.0
3398,2024,Johor Darul Ta'zim,Christian Abad,12,18,Goalkeeper,Defenders,3430.0


In [128]:
# OLDEST Player of all time
df_msl_player[df_msl_player['AGE'] == max_age.iloc[4]]

,SEASON,TEAM,PLAYER,NUMBER,AGE,POSITION,POSITION_TYPE,VALUE_EUR
2732,2022,Kuching City,Ramesh,12,41,Left-Back,Defenders,2748.0


In [129]:
# OLDEST Player in 2024
oldest_2024 = season_2024.loc[season_2024['AGE'].idxmax()]

season_2024[season_2024['AGE'] == oldest_2024[4]]

#df_msl_player[df_msl_player['AGE'] == min_age.iloc[4]]

C:\Users\izzat\AppData\Local\Temp\ipykernel_19808\3617138358.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  season_2024[season_2024['AGE'] == oldest_2024[4]]


,SEASON,TEAM,PLAYER,NUMBER,AGE,POSITION,POSITION_TYPE,VALUE_EUR
2985,2024,Johor Darul Ta'zim,Farizal Marlias,1,38,Goalkeeper,Goalkeeper,100000.0
3004,2024,Johor Darul Ta'zim,Natxo Insa,30,38,Central Midfield,Midfielders,75000.0
3231,2024,Kuching City,Wan Azraie,38,38,Goalkeeper,Goalkeeper,50000.0


In [130]:
# Average player's age per team in every season

pivot_msl = df_msl_player.pivot_table(values='AGE', index='SEASON', columns='TEAM', aggfunc='mean').fillna(value=0).round(0)

team_2024 = ["""Johor Darul Ta'zim""" ,'Selangor FC', 'Terengganu FC', 'Kedah Darul Aman FC' ,'Sri Pahang FC' ,'Kelantan Darul Naim' ,'Kuala Lumpur City FC' ,'Kuching City' ,'Negeri Sembilan FC' ,'PDRM FC' ,'Penang FC' ,'Perak FC' ,'Sabah FC' ]

pivot_msl_2024 = pivot_msl.filter(items=team_2024)
pivot_msl_2024

,Johor Darul Ta'zim,Selangor FC,Terengganu FC,Kedah Darul Aman FC,Sri Pahang FC,Kelantan Darul Naim,Kuala Lumpur City FC,Kuching City,Negeri Sembilan FC,PDRM FC,Penang FC,Perak FC,Sabah FC
SEASON,,,,,,,,,,,,,
2016,25.0,24.0,0.0,22.0,24.0,0.0,0.0,0.0,0.0,0.0,24.0,24.0,0.0
2017,26.0,24.0,26.0,24.0,25.0,0.0,25.0,0.0,25.0,0.0,0.0,25.0,0.0
2018,25.0,25.0,26.0,24.0,26.0,0.0,26.0,0.0,0.0,0.0,0.0,25.0,0.0
2019,25.0,24.0,25.0,24.0,25.0,0.0,0.0,0.0,0.0,26.0,0.0,24.0,26.0
2020,26.0,23.0,24.0,25.0,25.0,0.0,26.0,0.0,0.0,0.0,24.0,24.0,25.0
2021,27.0,23.0,24.0,25.0,25.0,0.0,24.0,0.0,26.0,0.0,25.0,0.0,26.0
2022,27.0,23.0,24.0,25.0,27.0,26.0,25.0,27.0,26.0,26.0,24.0,22.0,27.0
2024,29.0,25.0,27.0,26.0,28.0,26.0,28.0,28.0,26.0,27.0,26.0,26.0,28.0


In [132]:
# Average player's value per team in every season

pivot_msl = df_msl_player.pivot_table(values='VALUE_EUR', index='SEASON', columns='TEAM', aggfunc='mean').fillna(value=0).round(0)

team_2024 = ["""Johor Darul Ta'zim""" ,'Selangor FC', 'Terengganu FC', 'Kedah Darul Aman FC' ,'Sri Pahang FC' ,'Kelantan Darul Naim' ,'Kuala Lumpur City FC' ,'Kuching City' ,'Negeri Sembilan FC' ,'PDRM FC' ,'Penang FC' ,'Perak FC' ,'Sabah FC' ]

pivot_msl_2024 = pivot_msl.filter(items=team_2024)
pivot_msl_2024

,Johor Darul Ta'zim,Selangor FC,Terengganu FC,Kedah Darul Aman FC,Sri Pahang FC,Kelantan Darul Naim,Kuala Lumpur City FC,Kuching City,Negeri Sembilan FC,PDRM FC,Penang FC,Perak FC,Sabah FC
SEASON,,,,,,,,,,,,,
2016,170846.0,40122.0,0.0,58424.0,38020.0,0.0,0.0,0.0,0.0,0.0,34448.0,12873.0,0.0
2017,244366.0,14733.0,16896.0,56318.0,46941.0,0.0,30341.0,0.0,35396.0,0.0,0.0,30850.0,0.0
2018,240029.0,52966.0,59160.0,62212.0,118589.0,0.0,50032.0,0.0,0.0,0.0,0.0,78985.0,0.0
2019,213113.0,64152.0,74990.0,68532.0,72924.0,0.0,0.0,0.0,0.0,23519.0,0.0,56671.0,17683.0
2020,233566.0,110657.0,72265.0,103758.0,98193.0,0.0,103562.0,0.0,0.0,0.0,42613.0,101936.0,62176.0
2021,247693.0,79753.0,63367.0,63796.0,184419.0,0.0,98251.0,0.0,100096.0,0.0,51007.0,0.0,80278.0
2022,293445.0,156140.0,110112.0,132428.0,122356.0,63382.0,110377.0,39823.0,83167.0,25737.0,70810.0,37263.0,119283.0
2024,300873.0,184240.0,154945.0,105296.0,146154.0,57965.0,112895.0,73524.0,80231.0,64694.0,76860.0,108282.0,111190.0
